In [1]:
import time
import random
import copy

In [2]:
def get_best_set(sets,element_coverage,cost,k,selected_ids):
        cost_per_cover = []
        for j in range(len(sets)):
            coverage = sum(1 for element in sets[j] if element_coverage[element] < k)
            if coverage !=0:
                cost_per_cover.append([cost[j]/coverage,coverage])
            else:
                cost_per_cover.append([10000000,coverage])
        
        best_set_indeks = 0
        best_set_cover = 10000000000 
        for p in range(len(sets)):
             if selected_ids[p]!=1:
                if cost_per_cover[p][0]<best_set_cover:
                        best_set_indeks = p
                        best_set_cover = cost_per_cover[p][0]
                elif cost_per_cover[p][0]==best_set_cover:
                    if cost_per_cover[p][1]>cost_per_cover[best_set_indeks][1]:
                            best_set_indeks = p
                            best_set_cover = cost_per_cover[p][0] 
        return best_set_indeks

In [3]:
class Individual:
    
    def __init__(self, n):
        self.genes = []
        self.obj = 0
        for i in range(n):
            gene = random.randint(0, 1)
            self.genes.append( gene )

    def generateGreedy(self, universe, sets, c, k):
        universe = list(set(universe))
        element_coverage = {element: 0 for element in universe}
        selected_sets = []
        selected_ids = [0 for element in sets]
        selected_cost = 0
        while any(coverage < k for coverage in element_coverage.values()):
            best_set_id = get_best_set(sets,element_coverage,c,k,selected_ids)
            best_set = sets[best_set_id]
            for element in best_set:
                element_coverage[element] += 1
            selected_ids[best_set_id] = 1
            selected_sets.append(sets[best_set_id])
            selected_cost += c[best_set_id]         
        self.genes = selected_ids
        self.obj = selected_cost
       

    def fitness(self,cost):
        value = 0
        for k in range(len(self.genes)):
            if self.genes[k] == 1:
                value += cost[k]
        self.obj = value

    def validate(self,universe, sets, k):
        element_coverage = {element: 0 for element in universe}
        for i in range(len(self.genes)):
            if self.genes[i] == 1:
                for p in sets[i]:
                    element_coverage[p] += 1
        for i in element_coverage:
            if element_coverage[i] < k:
                return False
        return True
    

In [4]:
class VNS:


    def __init__(self, universe, sets, cost, k, out_path):
        self.out_path = out_path
        self.vns_min = 2
        self.vns_max = 10
        self.time_limit = 600
        self.no_impr_iterations = 10000000
        self.iterations_max = 10000000000
        self.iteration_max_found = 0
        self.universe = universe
        self.sets = sets
        self.cost = cost
        self.k = k

    def local_search_BestImprovementStrategy(self, x):
        has_better = True
        while has_better:
            has_better = False
            ind_best = Individual(len(self.sets))
            ind_best.genes = []
            ind_best.genes = copy.deepcopy(x.genes)
            ind_best.obj = x.obj

            for i in range(len(self.sets)):
                genes_copy = []
                genes_copy =  copy.deepcopy(x.genes)

                if genes_copy[ i ] == 1:
                    genes_copy[ i ] = 0
                else:
                    genes_copy[ i ] = 1
                
                x_prime = Individual(len(self.sets))
                x_prime.genes = copy.deepcopy(genes_copy)
                if x_prime.validate(self.universe,self.sets,self.k):
                    x_prime.fitness(self.cost)
                    if(x_prime.obj < ind_best.obj):
                        ind_best.genes = []
                        ind_best.genes  = copy.deepcopy(genes_copy)
                        ind_best.obj = x_prime.obj
                        has_better = True
            if has_better :
                x.genes = []
                x.genes = copy.deepcopy(ind_best.genes)
                x.obj   = ind_best.obj
                del ind_best
    
    def local_search_FirstImprovementStrategy(self, x):
        for i in range(len(self.sets)):
            genes_copy = []
            genes_copy =  copy.deepcopy(x.genes)
            if genes_copy[ i ] == 1:
                genes_copy[ i ] = 0
            else:
                genes_copy[ i ] = 1
            x_prime = Individual(len(self.sets))
            x_prime.genes = copy.deepcopy(genes_copy)
            if x_prime.validate(self.universe,self.sets,self.k):
                x_prime.fitness(self.cost)
                if(x_prime.obj < x.obj):
                    x.genes = []
                    x.genes  = copy.deepcopy(genes_copy)
                    x.obj = x_prime.obj
                    break

    def shake(self, x, vns_shake):
        x_p = copy.deepcopy(x.genes)

        for i in range(vns_shake):
            ix = random.randint(0, len(x.genes) - 1)
            if x_p[ ix ] == 1:
                x_p[ ix ] = 0
            else:
                x_p[ ix ] = 1
        
        return x_p

    def run(self):
        x = Individual(len(self.sets))
        x.generateGreedy(self.universe,self.sets.copy(),self.cost,self.k)
        print(x.obj)
        self.vns_min = 2
        self.vns_max = len(x.genes)//3
        iterations_num = 0
        no_impr = 0
        start_time = time.time()
        end_time = time.time()

        while end_time - start_time < self.time_limit and no_impr < self.no_impr_iterations and iterations_num < self.iterations_max:
            vns_param = self.vns_min
            while vns_param <= self.vns_max and end_time-start_time < self.time_limit:
                xk_genes = copy.deepcopy(self.shake(x, vns_param))
                x_prime = Individual(len(self.sets))
                x_prime.genes = []
                x_prime.genes = copy.deepcopy(xk_genes)
                x_prime.fitness(self.cost)
                #self.local_search_BestImprovementStrategy(x_prime)
                self.local_search_FirstImprovementStrategy(x_prime)

                if(x_prime.obj < x.obj and x_prime.validate(self.universe,self.sets,self.k)): # new best found
                    x.genes = []
                    x.genes = copy.deepcopy(x_prime.genes)
                    x.obj = x_prime.obj
                    vns_param = self.vns_min
                    print("New best solution found: ", x.obj)
                    self.iteration_max_found = iterations_num + 1
                    no_impr = 0
                else:
                    vns_param = vns_param + 1
                    no_impr = no_impr + 1
                iterations_num += 1
                end_time = time.time()
        print(x.obj)
        valid = x.validate(self.universe,self.sets,self.k)

        if valid == False:
            print("It is not a valid solution")
        else:
            print("We found a valid solution")
            f = open(self.out_path, "w")
            f.write("Cijena:\n" + str(x.obj) + "\n")
            #f.write("Rezultat:\n" + str(x.genes) + "\n")
            f.write("Vrijeme: " + str(end_time-start_time))
            f.close()

In [5]:
def get_input_from_file(input_file):
    f = open(input_file, "r")
    pomocna = f.readline()
    pomocna = pomocna.split()
    n = int(pomocna[0]) #br univerzuma
    m = int(pomocna[1]) #br podskupova

    c = [] #cijena j-og skupa
    while len(c)<m:
        pomocna = f.readline()
        pomocna = pomocna.split()
        for j in pomocna:
            c.append(int(j))
    
    d = {}
    for i in range(n):
        pomocna = f.readline()
        k = int(pomocna)
        pomocna3 = []
        while len(pomocna3)<k:
            pomocna = f.readline()
            pomocna = pomocna.split()
            for j in pomocna:
                pomocna3.append(int(j))
        d[i] = [ k, pomocna3.copy()]

    rezultat = []
    rezultat.append(n)
    rezultat.append(m)
    rezultat.append(c.copy())
    rezultat.append(d)
    return rezultat

In [6]:
def create_universe(size):
    universe = []
    for k in range(size):
        universe.append(k + 1)
    universe = set(universe)
    return universe

In [7]:
def create_sets(cover,n,m):
    sets = [[] for k in range(m)]
    for i in range(n):
        for element in cover[i][1]:
            sets[element-1].append(i+1)
    return sets

In [8]:
def find_k_max(cover,n):
    result = 1000000000000000
    for k in range(n):
        if cover[k][0]<result:
            result = cover[k][0]
    return result

In [9]:
lista = ["scp01.txt"]
for j in lista:
    input_file = j
    data = get_input_from_file("instance\\"+input_file)
    n = data[0]
    m = data[1]
    cost = data[2]
    cover = data[3]

    sets = create_sets(cover,n,m)
    universe = create_universe(n)
    k_min = 2
    k_max = find_k_max(cover,n)
    k_med = (k_min+k_max)//2
    
    for k in range(5):
        universe2 = universe.copy()
        sets2 = sets.copy()
        cost2 = cost.copy()
        vns = VNS(universe2,sets2,cost2,k_max,"rjesenja_k_max\VNS\VNS_"+str(k)+ "_" + input_file)
        vns.run()   

388
388
We found a valid solution
388
388
We found a valid solution
388
388
We found a valid solution
388
New best solution found:  387
New best solution found:  386
386
We found a valid solution
388
388
We found a valid solution
386
386
We found a valid solution
386
New best solution found:  385
385
We found a valid solution
386
386
We found a valid solution
386
386
We found a valid solution
386
New best solution found:  385
New best solution found:  384
384
We found a valid solution
414
New best solution found:  411
New best solution found:  410
410
We found a valid solution
414
414
We found a valid solution
414
414
We found a valid solution
414
New best solution found:  413
413
We found a valid solution
414
414
We found a valid solution
438
438
We found a valid solution
438
438
We found a valid solution
438
438
We found a valid solution
438
438
We found a valid solution
438
438
We found a valid solution
416
416
We found a valid solution
416
New best solution found:  415
415
We found